# 06. MySQL_to_BigQuery(Using_Embulk)
- 본 문서에서는 MySQL에서 BigQuery로 데이터를 로딩하는 과정을 설명합니다
- [Embulk](https://github.com/embulk/embulk)는 다양한 종류의 데이터 소스에서 데이터를 읽어 로딩을 해주게 해줍니다
- 병렬 로딩이 가능해 속도를 빠르게 할 수 있습니다
- 입력 데이터를 기반해 자동으로 스키마를 예측합니다

---

## 1) Embulk Install
- MacOS에서 진행했습니다
(JAVA가 설치되어 있지 않은 우분투라면 아래부터 실행해주세요!)
~~~
sudo apt-get update
sudo apt-get install default-jre
~~~
### Embulk 설치
~~~
curl --create-dirs -o ~/.embulk/bin/embulk -L "https://dl.embulk.org/embulk-latest.jar"
chmod +x ~/.embulk/bin/embulk
echo 'export PATH="$HOME/.embulk/bin:$PATH"' >> ~/.bashrc
source ~/.bashrc
~~~
-  ~/.embulk 디렉토리에 설치가 되었으며, embulk -version을 통해 version을 확인할 수 있습니다

### Embulk Plugin 설치
~~~
embulk gem install [plugin name]
~~~
Plugin은 [여기](http://www.embulk.org/plugins/)를 클릭하시면 리스트를 볼 수 있습니다!
input으로 mysql, output으로 bigquery를 사용하기 위해 플러그인을 설치해보겠습니다
~~~
embulk gem install embulk-input-mysql
embulk gem install embulk-output-bigquery
~~~
embulk gem list로 설치된 플러그인을 확인할 수 있습니다


### Embulk 기본 명령어 설명
- embulk는 yml 파일에 설정을 저장하고 이를 이용해 작업을 실행합니다

~~~
embulk example
~~~
    - 예제 데이터를 생성
    
~~~
embulk guess embulk-example/seed.yml 
~~~
    - seed.yml파일을 기반으로 설정을 추론합니다
    
~~~
embulk guess embulk-example/seed.yml -o config.yml
~~~
    - seed.yml파일을 기반으로 설정파일인 config.yml을 생성합니다

~~~
embulk preview config.yml
~~~
    - config.yml을 읽어 데이터를 파싱이 진행되는지 테스트합니다
    
~~~
embulk run config.yml
~~~
    - config.yml을 읽어 실행합니다
---

## 2) MySQL to BigQuery

- in, out에 대한 파라미터는 아래 링크를 클릭해주시면 연결됩니다! ( 꼭 자세히 읽어보세요! )
- [input-mysql](https://github.com/embulk/embulk-input-jdbc/tree/master/embulk-input-mysql), [output-bigquery](https://github.com/embulk/embulk-output-bigquery)
- Example에서 제공하는 소스를 가지고 왔습니다!
- json key file을 설정하는 방법은 [링크](https://github.com/zzsza/bigquery-tutorial/blob/master/tutorials/05-ETC/01.%20GOOGLE_CLOUD_CRENDENTIALS_json_file_setting.ipynb)를 클릭해주세요!

### yml 파일
~~~
in:
    type: mysql
    host: localhost
    port: 3306
    user: root   
    password: root    
    database: database_name 
    table: table_name  
    select: "col1, col2, col3"
    where: "col4 != 'a'"
    order_by: "col1 DESC"
out:
    type: bigquery
    mode: replace
    auth_method: json_key
    json_keyfile: /path/to/json_keyfile.json
    project: my-project   
    dataset: reservation   
    table: reservation   
    gcs_bucket: seongyun   
    auto_create_gcs_bucket: true 
    auto_create_table: true 
    auto_create_dataset: true
~~~


- 이런 방식으로 yml 파일을 저장합니다!

~~~
embulk run config.yml
~~~
- run 명령어를 토대로 embulk가 자동으로 gcs_bucket에 데이터를 csv로 저장한 후, load해줍니다

---
### 추가 설명
- in mysql에서 쿼리 결과를 넣고싶다면 아래와 같이 작성하시면 됩니다
~~~
in:
  type: mysql
  host: localhost
  user: root
  password: root
  database: database_name
  query: |
    SELECT t1.id, t1.name, t2.id AS t2_id, t2.name AS t2_name
    FROM table1 AS t1
    LEFT JOIN table2 AS t2
      ON t1.id = t2.t1_id
~~~
- column에 대해 option을 부여한다면 아래와 같이!
~~~
in:
  type: mysql
  host: localhost
  user: root
  password: root
  database: database_name
  table: table_name
  select: "col1, col2, col3"
  where: "col4 != 'a'"
  column_options:
    col1: {type: long}
    col3: {type: string, timestamp_format: "%Y/%m/%d", timezone: "+0900"}
  after_select: "update table_name set col5 = '1' where col4 != 'a'"
~~~

---
### Multi Thread 사용하기
- 위에 in, out으로 데이터를 넣을 경우 느리다면 [Multi Thread](http://www.embulk.org/docs/built-in.html#local-executor-plugin)를 사용해서 작업을 진행하면 됩니다!
- yml 파일 상단에 아래와 같이 작성해주시면 됩니다
~~~
exec:
    max_threads: 16
    min_output_tasks: 4
~~~